# [Aligning Big Brains and Atlases](https://biop.github.io/ijp-imagetoatlas/) in Python

This series of notebook demoes the use of ABBA with python.

If you have executed the previous notebook, you can re-open the saved state file.

This notebook shows how to convert the pixel position of a slice to its coordinates in the atlas (forth and back).


In [ ]:
# core dependencies
import os
import time
from pathlib import Path


from bg_atlasapi import show_atlases
from bg_atlasapi import utils

from abba_python.Abba import Abba

In [ ]:
# -- NOT HEADLESS
abba = Abba('Adult Mouse Brain - Allen Brain Atlas V3p1') # You may have to click a window!
abba.show_bdv_ui()  # creates and show a bdv view

save_dir = os.path.join(os.getcwd(), 'temp', 'notebook0', 'state') # execute the first notebook to get this file!
abba.state_load(save_dir+"/state.abba") # full absolute path needed

# ALL COMMANDS ARE PERFORMED ON SELECTED SLICES!!
# since we want to register all of them, we select all of them
abba.select_all_slices()

# change display settings : channel index, min, max
abba.change_display_settings(0, 0, 500)
abba.change_display_settings(1, 0, 1200)

# YOU MAY NEED TO CLICK ON THE UI TO FINISH THE EXECUTION OF THIS CELL (angle change notification)

In [ ]:
# programmatic way to show (or hide) sections and channels
abba.get_bdv_view().setSelectedSlicesVisibility(True)
abba.get_bdv_view().setSelectedSlicesVisibility(0, True)

In [ ]:
mp = abba.mp
mp.selectSlice(mp.getSlices()) # select all slices

In [ ]:
mp.deselectSlice(mp.getSlices()) # deselect all

In [ ]:
mp.selectSlice(mp.getSlices().get(2)) # select the third slice

In [ ]:
# The slices are always sorted from small z to high z. To keep track of who's who, reference them before moving them
slice_0 = mp.getSlices().get(0) 
slice_1 = mp.getSlices().get(1)
slice_2 = mp.getSlices().get(2)

In [ ]:

from jpype.types import JString, JArray, JDouble

# Get transformation
transform_pix_to_atlas = slice_0.getSlicePixToCCFRealTransform()

DoubleArray = JArray(JDouble)

coordInImage = DoubleArray(3)
coordInCCF = DoubleArray(3)

coordInImage[0] = 500 # X (pixel)
coordInImage[1] = 500 # Y
coordInImage[2] = 0 # Z

transform_pix_to_atlas.inverse().apply(coordInImage,coordInCCF)

print('CCF coord (mm):'+str(coordInCCF))


In [ ]:
# We can also find the coordinates of the atlas into a slice
# a coordinate in z far from 0, means that it's not in the slice, but far out
# See https://forum.image.sc/t/coordinates-in-slice-to-atlas-using-abba-python/79513 

coordInCCF = DoubleArray(3)
coordInCCF[0] = 6.60 # X (pixel)
coordInCCF[1] = 4.31 # Y
coordInCCF[2] = 5.58 # Z

coordInImage = DoubleArray(3)

for idx in range(0,mp.getSlices().size()):
    # Get transformation
    transform_pix_to_atlas = mp.getSlices().get(idx).getSlicePixToCCFRealTransform()
    transform_pix_to_atlas.apply(coordInCCF, coordInImage)
    print('Slice['+str(idx)+'] coord (pixel):'+str(coordInImage))
    


In [ ]:
# The slice indexed 6 should be the one which is the nearest to the atlas point.
# Note that the values in Z are small because, in absence of 3D image, 
# each section is assumed to have a size of 1 mm

In [ ]:
# You can normalize the slices sections by expanding them to cover the space between neighboring slices, no more, no less
abba.select_all_slices()
abba.set_slices_thickness_match_neighbors() # not critical, but for 3d reconstruction it will allow for each slice to occupy the place available between its neighbors
# The coordinates will be different in Z
coordInImage = DoubleArray(3)

for idx in range(0,mp.getSlices().size()):
    # Get transformation
    transform_pix_to_atlas = mp.getSlices().get(idx).getSlicePixToCCFRealTransform()
    transform_pix_to_atlas.apply(coordInCCF, coordInImage)
    print('Slice['+str(idx)+'] coord (pixel):'+str(coordInImage))